<a href="https://colab.research.google.com/github/tijazz/Big-Data/blob/main/Exploring_Postgres_for_Big_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Querying relational data with Postgres

In [1]:
# Install Postgres Server, and start server
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# avoid printing connections
con_py = "/usr/local/lib/python3.6/dist-packages/sql/connection.py"
!sed -i 's/ print(cls.connection_list())/ if len(cls.connections)>1:print(cls.connection_list())/' $con_py
# setup client with magic
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres


 * Starting PostgreSQL 10 database server
   ...done.
CREATE ROLE
sed: can't read /usr/local/lib/python3.6/dist-packages/sql/connection.py: No such file or directory


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

In [2]:
#Upload dummy data files (game-clicks.csv, buy-clicks.csv, ad-clicks.csv) to your colab session.  



In [3]:
#	Create buyclicks table and import the data

%%sql
CREATE TABLE buyclicks (
    timestamp TIMESTAMP WITHOUT TIME ZONE NOT NULL, 
    txid INTEGER NOT NULL, 
    usersessionid INTEGER NOT NULL, 
    team INTEGER NOT NULL, 
    userid INTEGER NOT NULL, 
    buyid INTEGER NOT NULL, 
    price FLOAT NOT NULL
);



 * postgresql+psycopg2://@/postgres


""


In [4]:
#will import the data from the CSV file 

%%sql
COPY buyclicks FROM '/content/buy-clicks.csv' DELIMITER ',' CSV HEADER;



 * postgresql+psycopg2://@/postgres


""


In [5]:
#Verify the data is imported: 
%sql select * from buyclicks


 * postgresql+psycopg2://@/postgres


,timestamp,txid,usersessionid,team,userid,buyid,price
0,2016-05-26 15:36:54,6004,5820,9,1300,2,3.0
1,2016-05-26 15:36:54,6005,5775,35,868,4,10.0
2,2016-05-26 15:36:54,6006,5679,97,819,5,20.0
3,2016-05-26 16:36:54,6067,5665,18,121,2,3.0
4,2016-05-26 17:06:54,6093,5709,11,2222,5,20.0
...,...,...,...,...,...,...,...
2942,2016-06-16 10:36:54,39838,34373,35,305,0,1.0
2943,2016-06-16 10:36:54,39839,37360,168,2016,1,2.0
2944,2016-06-16 10:36:54,39840,34542,64,68,2,3.0
2945,2016-06-16 10:36:54,39841,38277,27,667,4,10.0


In [6]:
#	Create gameclicks table and import the data: 

%%sql
CREATE TABLE gameclicks (
    timestamp TIMESTAMP WITHOUT TIME ZONE NOT NULL, 
    clickid INTEGER NOT NULL, 
    userid INTEGER NOT NULL, 
    usersessionid INTEGER NOT NULL, 
    isHit INTEGER NOT NULL, 
    teamid INTEGER NOT NULL, 
    teamLevel INTEGER NOT NULL
);


 * postgresql+psycopg2://@/postgres


""


In [7]:
#will import the data from the CSV file 

%%sql
COPY gameclicks FROM '/content/game-clicks.csv' DELIMITER ',' CSV HEADER;



 * postgresql+psycopg2://@/postgres


""


In [8]:
#Verify the data is imported: 
%sql select * from gameclicks


 * postgresql+psycopg2://@/postgres


,timestamp,clickid,userid,usersessionid,ishit,teamid,teamlevel
0,2016-05-26 15:06:55,105,1038,5916,0,25,1
1,2016-05-26 15:07:09,154,1099,5898,0,44,1
2,2016-05-26 15:07:14,229,899,5757,0,71,1
3,2016-05-26 15:07:14,322,2197,5854,0,99,1
4,2016-05-26 15:07:20,22,1362,5739,0,13,1
...,...,...,...,...,...,...,...
755801,2016-06-16 11:06:39,755047,664,38792,1,84,8
755802,2016-06-16 11:06:44,755006,2374,34634,1,72,8
755803,2016-06-16 11:06:46,755521,1402,33063,1,141,6
755804,2016-06-16 11:06:50,755592,290,37357,1,146,6


In [9]:
#	Create adclicks table and import the data
%%sql
CREATE TABLE adclicks (
    timestamp TIMESTAMP WITHOUT TIME ZONE NOT NULL, 
    txid INTEGER NOT NULL, 
    usersessionid INTEGER NOT NULL, 
    teamid INTEGER NOT NULL, 
    userid INTEGER NOT NULL, 
    adid INTEGER NOT NULL, 
    adcategory VARCHAR(11) NOT NULL
);


 * postgresql+psycopg2://@/postgres


""


In [10]:
#will import the data from the CSV file 

%%sql
COPY adclicks FROM '/content/ad-clicks.csv' DELIMITER ',' CSV HEADER;



 * postgresql+psycopg2://@/postgres


""


In [11]:
#Verify the data is imported: 
%sql select * from adclicks


 * postgresql+psycopg2://@/postgres


,timestamp,txid,usersessionid,teamid,userid,adid,adcategory
0,2016-05-26 15:13:22,5974,5809,27,611,2,electronics
1,2016-05-26 15:17:24,5976,5705,18,1874,21,movies
2,2016-05-26 15:22:52,5978,5791,53,2139,25,computers
3,2016-05-26 15:22:57,5973,5756,63,212,10,fashion
4,2016-05-26 15:22:58,5980,5920,9,1027,20,clothing
...,...,...,...,...,...,...,...
16318,2016-06-16 11:04:02,39800,34697,82,1214,16,clothing
16319,2016-06-16 11:04:05,39807,34673,77,751,23,fashion
16320,2016-06-16 11:04:12,39832,34262,18,1328,11,games
16321,2016-06-16 11:04:33,39794,34287,22,415,15,sports


In [12]:
#Filter rows and columns. We can query only the price and userid columns with the following command:
%%sql
select price, userid from buyclicks;


 * postgresql+psycopg2://@/postgres


,price,userid
0,3.0,1300
1,10.0,868
2,20.0,819
3,3.0,121
4,20.0,2222
...,...,...
2942,1.0,305
2943,2.0,2016
2944,3.0,68
2945,10.0,667


In [13]:
%%sql
select price, userid from buyclicks where price > 10;


 * postgresql+psycopg2://@/postgres


,price,userid
0,20.0,819
1,20.0,2222
2,20.0,1304
3,20.0,1027
4,20.0,1544
...,...,...
605,20.0,212
606,20.0,2209
607,20.0,1546
608,20.0,221


#Aggregate SQL Functions

In [15]:
%%sql
select avg(price) from buyclicks;


 * postgresql+psycopg2://@/postgres


,avg
0,7.263997


In [16]:
%sql select sum(price) from buyclicks;

 * postgresql+psycopg2://@/postgres


,sum
0,21407.0


#Combining Tables

In [17]:
%%sql
select adid, buyid, adclicks.userid
from adclicks join buyclicks on adclicks.userid = buyclicks.userid;


 * postgresql+psycopg2://@/postgres


,adid,buyid,userid
0,2,5,611
1,2,4,611
2,2,4,611
3,2,5,611
4,2,4,611
...,...,...,...
103151,15,3,415
103152,0,1,2016
103153,0,4,2016
103154,0,0,2016


In [29]:
%%sql

select adcategory, count(adid) from adclicks
GROUP BY adcategory 
ORDER BY count(adid) DESC


 * postgresql+psycopg2://@/postgres


,adcategory,count
0,computers,2638
1,games,2601
2,clothing,2340
3,sports,2074
4,fashion,1727
5,movies,1692
6,hardware,1588
7,electronics,1097
8,automotive,566


In [33]:
%%sql 
select * from buyclicks
where price >= 20;

 * postgresql+psycopg2://@/postgres


,timestamp,txid,usersessionid,team,userid,buyid,price
0,2016-05-26 15:36:54,6006,5679,97,819,5,20.0
1,2016-05-26 17:06:54,6093,5709,11,2222,5,20.0
2,2016-05-26 17:06:54,6094,5798,77,1304,5,20.0
3,2016-05-26 18:06:54,6155,5920,9,1027,5,20.0
4,2016-05-26 18:36:54,6183,5893,64,1544,5,20.0
...,...,...,...,...,...,...,...
605,2016-06-16 09:36:54,39708,34524,63,212,5,20.0
606,2016-06-16 09:36:54,39712,35141,104,2209,5,20.0
607,2016-06-16 09:36:54,39714,34703,82,1546,5,20.0
608,2016-06-16 10:06:54,39783,34325,27,221,5,20.0
